# SealNet Detection Pipeline -- Training / Validation
---

This jupyter notebook will go through training CNN within the SealNet pipeline. Requires a training set generated on the [training_set_generation](https://github.com/iceberg-project/Seals/blob/paper/SealNet_code/training_set_generation.ipynb) notebook.




## Table of contents
---
* [Getting started](#intro)
    * [Setup](#setup)
    * [Visualize training set](#vis_imgs)
* [Pipeline 1 - Heatmap](#1)
    * [Training](#1T)
    * [Validation](#1V)
* [Pipeline 2.1 - Heatmap + count](#2.1)
    * [Training](#2.1T)
    * [Validation](#2.1V)
* [Pipeline 2.2 - Heatmap + occupancy](#2.2)
    * [Training](#2.2T)
    * [Validation](#2.2V)
* [Pipeline 3 - Heatmap + count + occupancy](#3)
    * [Training](#3T)
    * [Validation](#3V)

## Getting started<a name="intro"></a>
---

If you followed the *training_set_generation* jupyter notebook (also present in this repo), you should have training sets generated and hyperparameter sets to try out, and be ready to search for a best performing seal detection pipeline.  Output files in this repository are organized as follows: *'./{dest_folder}/{pipeline}/{model_settings}/{model_settings}_{file}'*

### Setup environment<a name="setup"></a>

Before training and validating model/hyperparameter combinations inside the pipelines, we need to load the required python modules and a few global variables.

In [14]:
# import required packages
import os
import rasterio
import pandas as pd
import numpy as np
import operator
from PIL import Image 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
from functools import reduce
from utils.model_library import * 

%matplotlib inline
mpl.rcParams['figure.dpi']= 400

# destination folder for saved models and model stats
dest_folder = 'saved_models'

### Visualizing training images (Optional)<a name="vis_imgs"></a>

To get a better sense for what the training set is like, the next cell will display a few random images from the training classes. Displayed images are extracted from a pool of ~75000 training images. 

In [ ]:
# save class names
class_names = sorted([subdir for subdir in os.listdir('./training_sets/training_set_vanilla/training')])
# store images
images = []

# loop over labels
for label in class_names:
    for path, _, files in os.walk('./training_sets/training_set_vanilla/training/{}'.format(label)):
        files = np.random.choice(files, 5)
        for filename in files:
            images.append(np.asarray(Image.open(os.path.join(path, filename))))

images = np.array(images)

# display images 
ncols=len(class_names)
nindex, height, width, intensity = images.shape
nrows = nindex // ncols
# check if rows and columns can fit the number of images
assert nindex == nrows * ncols
result = (images.reshape(nrows, ncols, height, width, intensity)
          .swapaxes(1,2)
          .reshape(height*nrows, width*ncols, intensity))

plt.imshow(result)
cur_axes = plt.gca()
cur_axes.axes.get_xaxis().set_visible(False)
cur_axes.axes.get_yaxis().set_visible(False)
plt.show()
    


## Pipeline 1 - Heatmap models <a name="1"></a>
---

Heatmap models work by using semantic segmentation to generate a pixel-wise probability of a cell being a seal centroid. With a matrix of probabilities, we obtain a count by applying a sigmoid transform to it, thresholding and adding over all cells. Seal locations on an image are determined by finding the *n* greatest peaks of intensity in the image, where *n* is the count for that image.


### Training<a name="1T"></a>

The following cell trains heatmap based models on a selected combination of architectures, training sets and hyperparameters. For valid entries, see "*/utils/model_library.py*" within the main repo.

In [15]:
# switch pipeline
pipeline = 'Heatmap'

# generate model combinations
combinations_1 = {'model_architecture': ['Unet'],
                  'training_dir': ['training_set_vanilla'],
                  'hyperparameter_set': ['D']}

# read as a DataFrame
combinations_1 = pd.DataFrame(combinations_1)
                    
# create folders for resulting files
for row in combinations_1.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]              
    if not os.path.exists("./{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("{}/{}/{}".format(dest_folder, pipeline, mdl)) 


We can then provide model combinations created above as arguments to the training script, *train_sealnet.py*. A list of required arguments can be displayed by running the cell below.

In [ ]:
%run train_sealnet.py -h

In [16]:
# iterate over combinations
for row in combinations_1.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                             --hyperparameter_set=$hyp_st \
                             --output_name=$out --models_folder=$dest_folder
                              
      


training Unet_ts-vanilla

Epoch 1/75
----------

training 



### Validation<a name="1V"></a>

This script validates models from training, generating .csvs with precision and recall on the validation set. 

In [1]:
# iterate over combinations
for row in combinations_1.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is trained
    if "{}.tar".format(out) not in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was not trained'.format(out))
        continue
    
    print()
    !echo validating $out
    print()
    
    # run training
    !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                --hyperparameter_set=$hyp_st \
                                --output_name=$out --models_folder=$dest_folder

NameError: name 'combinations_1' is not defined

## Pipeline 2.1 - Heatmap + count<a name="1.1"></a>
---

This pipeline will also generate a seal intensity heatmap like the previous one, but it does not add over pixels to get a count. Instead, Heatmap + count models have a specialized branch to get a count by regression. Seal locations on an image are determined by finding the *n* greatest peaks of intensity in the image, where *n* is the count for that image

### Training<a name="2.1T"></a>

The following cell trains heatmap + count based models on a selected combination of architectures, training sets and hyperparameters. For valid entries, see "*/utils/model_library.py*" within the main repo.

In [17]:
# switch pipeline
pipeline = 'Heatmap-Cnt'

# generate model combinations
combinations_21 = {'model_architecture': ['UnetCntWRN'],
                   'training_dir': ['training_set_vanilla'],
                   'hyperparameter_set': ['D']}       

# read as a DataFrame
combinations_21 = pd.DataFrame(combinations_21)
                    

# create folders for resulting files
for row in combinations_21.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]               
    if not os.path.exists("{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("{}/{}/{}".format(dest_folder, pipeline, mdl)) 

In [ ]:
# iterate over combinations
for row in combinations_21.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                             --hyperparameter_set=$hyp_st \
                             --output_name=$out --models_folder=$dest_folder
                              
      

### Validation<a name="2.1V"></a>

This script validates models from training, generating .csvs with precision and recall on the validation set. 

In [ ]:
# iterate over combinations
for row in combinations_21.iterrows():
    
   # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is trained
    if "{}.tar".format(out) not in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was not trained'.format(out))
        continue
    
    print()
    !echo validating $out
    print()
    
    # run training
    !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                --hyperparameter_set=$hyp_st \
                                --output_name=$out --models_folder=$dest_folder

## Pipeline 2.2 - Heatmap + occupancy <a name="1.2"></a>
---

Heatmap + occupancy models, like pure Heatmap models, work by using semantic segmentation to generate a pixel-wise probability of a cell being a seal centroid. With a matrix of probabilities, we obtain a count by applying a sigmoid transform to it, thresholding and adding over all cells. After addining up cells, however, we use the output from a specialized occupancy branch -- and another threshold -- to decide if predicted counts will be set to zero (i.e. image is not occupied). Seal locations on an image are determined by finding the *n* greatest peaks of intensity in the image, where *n* is the count for that image.

### Training<a name="2.2T"></a>

The following cell trains heatmap + occupancy based models on a selected combination of architectures, training sets and hyperparameters. For valid entries, see "*/utils/model_library.py*" within the main repo.

In [ ]:
# switch pipeline
pipeline = 'Heatmap-Occ'

# generate model combinations
combinations_22 = {'model_architecture': ['UnetOccDense'],
                   'training_dir': ['training_set_vanilla'],
                   'hyperparameter_set': ['D'] }       

# read as a DataFrame
combinations_22 = pd.DataFrame(combinations_22)
                    

# create folders for resulting files
for row in combinations_22.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]                  
    if not os.path.exists("{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("{}/{}/{}".format(dest_folder, pipeline, mdl)) 

In [ ]:
# iterate over combinations
for row in combinations_22.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        #continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                             --hyperparameter_set=$hyp_st --output_name=$out  \
                             --models_folder=$dest_folder
   

### Validation<a name="2.2V"></a>

This script validates models from training, generating .csvs with precision and recall on the validation set. 

In [ ]:
# iterate over combinations
for row in combinations_22.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is trained
    if "{}.tar".format(out) not in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was not trained'.format(out))
        continue
    
    print()
    !echo validating $out
    print()
    
    # run validation
    !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                --hyperparameter_set=$hyp_st \
                                --output_name=$out --models_folder=$dest_folder

# Pipeline 3 - Heatmap + Count + Occupancy<a name="1.3T"></a>

Like all previous ones, this pipeline will also generate a seal intensity heatmap. Similar to Heatmap + count models, models on this pipeline have a specialized branch to get a count by regression. After counting, however, we use a threshold and the output from a specialized occupancy branch to decide if we will set the count to zero (i.e. image is not occupied). Seal locations on an image are determined by finding the *n* greatest peaks of intensity in the image, where *n* is the count for that image


### Training<a name="3T"></a>

The following cell trains heatmap + count + occupancy based models on a selected combination of architectures, training sets and hyperparameters. For valid entries, see "*/utils/model_library.py*" within the main repo.

In [12]:
# switch pipeline
pipeline = 'Heatmap-Cnt-Occ'

# generate model combinations
combinations_3 = {'model_architecture': ['UnetCntWRNOccDense'],
                   'training_dir': ['training_set_vanilla'],
                   'hyperparameter_set': ['D'] }       

# read as a DataFrame
combinations_3 = pd.DataFrame(combinations_3)
                    

# create folders for resulting files
for row in combinations_3.iterrows():
    mdl = row[1]['model_architecture'] + '_ts-' + row[1]['training_dir'].split('_')[-1]                  
    if not os.path.exists("{}/{}/{}".format(dest_folder, pipeline, mdl)):
        os.makedirs("{}/{}/{}".format(dest_folder, pipeline, mdl)) 

In [13]:
# iterate over combinations
for row in combinations_3.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('./{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was already trained'.format(out))
        #continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                             --hyperparameter_set=$hyp_st --output_name=$out  \
                             --models_folder=$dest_folder
   


training UnetCntWRNOccDense_ts-vanilla

Epoch 1/75
----------

training 

heatmap loss: 2.3010823183406903
count loss: 0.45830215521716555
occupancy loss: 0.7644140137030223

validation 

heatmap loss: 0.5803011048680082
count loss: 0.2466599441661196
occupancy loss: 0.3527543436242526
training time: 0.0h 15m 37s

Epoch 2/75
----------

training 

heatmap loss: 0.7834362572332068
count loss: 0.36345044802402426
occupancy loss: 0.4758972249235825

validation 

heatmap loss: 0.33263532237665133
count loss: 0.2201538124290163
occupancy loss: 0.7488286396566401
training time: 0.0h 31m 11s

Epoch 3/75
----------

training 

heatmap loss: 0.6611745876683544
count loss: 0.3045333038118967
occupancy loss: 0.36930637492276613

validation 

heatmap loss: 0.29965631509554996
count loss: 0.1687900461457432
occupancy loss: 0.34506139076526554
training time: 0.0h 46m 57s

Epoch 4/75
----------

training 

heatmap loss: 0.6439267598760787
count loss: 0.26024411624937704
occupancy loss: 0.32825351052

heatmap loss: 0.28013965547277614
count loss: 0.06262725355967949
occupancy loss: 0.12958461057405313
training time: 7.0h 44m 12s

Epoch 31/75
----------

training 

heatmap loss: 0.6347078735313967
count loss: 0.08872572081749928
occupancy loss: 0.14829044628497626

validation 

heatmap loss: 0.2804313989107592
count loss: 0.03491514248606648
occupancy loss: 0.08339769048596676
training time: 7.0h 59m 48s

Epoch 32/75
----------

training 

heatmap loss: 0.6322992704575037
count loss: 0.08986886661849378
occupancy loss: 0.14636566284814712

validation 

heatmap loss: 0.27996501508702726
count loss: 0.049362481640241845
occupancy loss: 0.041811216103794666
training time: 8.0h 15m 22s

Epoch 33/75
----------

training 

heatmap loss: 0.6365875063410316
count loss: 0.08735451713763144
occupancy loss: 0.1379193772333371

validation 

heatmap loss: 0.28026199377094224
count loss: 0.037586181451704816
occupancy loss: 0.04705962745232401
training time: 8.0h 30m 58s

Epoch 34/75
----------

t

heatmap loss: 0.631667833090941
count loss: 0.06760439341084641
occupancy loss: 0.12157173480037574

validation 

heatmap loss: 0.27989187500607005
count loss: 0.03947690111928676
occupancy loss: 0.16284791938948726
training time: 15.0h 30m 59s

Epoch 61/75
----------

training 

heatmap loss: 0.6309018174922515
count loss: 0.06676017709963245
occupancy loss: 0.11121211752414835

validation 

heatmap loss: 0.27966567064667225
count loss: 0.026796972195153798
occupancy loss: 0.08945187990587859
training time: 15.0h 46m 33s

Epoch 62/75
----------

training 

heatmap loss: 0.6352007173089391
count loss: 0.06838139458362069
occupancy loss: 0.11013147122438867

validation 

heatmap loss: 0.2814122343507411
count loss: 0.026826698655835687
occupancy loss: 0.1288429330919323
training time: 16.0h 2m 5s

Epoch 63/75
----------

training 

heatmap loss: 0.6189938414381942
count loss: 0.06623604103551359
occupancy loss: 0.11722928369976882

validation 

heatmap loss: 0.28038966299475804
count lo

### Validation<a name="3V"></a>

This script validates models from training, generating .csvs with precision and recall on the validation set. 

In [ ]:
# iterate over combinations
for row in combinations_3.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st = row[1]['training_dir'], row[1]['model_architecture'], \
                          row[1]['hyperparameter_set']
    out = arch + '_ts-' + t_dir.split('_')[-1]
    
    # check if model is trained
    if "{}.tar".format(out) not in os.listdir('{}/{}/{}/'.format(dest_folder, pipeline, out)): 
        print('{} was not trained'.format(out))
        continue
    
    print()
    !echo validating $out
    print()
    
    # run validation
    !python validate_sealnet.py --training_dir=$t_dir --model_architecture=$arch \
                                --hyperparameter_set=$hyp_st \
                                --output_name=$out --models_folder=$dest_folder